# Titanic with pyspark

This exercise is an example to use machine learning methods for predicting survivors in the titanic dataset.

Referred heavily from : https://www.kaggle.com/code/roshan77/pyspark-classification-model

# Install pyspark and Download data

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 27.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b3e3c5c90e0be435e50519ab440274cc3394e1df200252b5df4a84f8c5bf4660
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
!wget https://raw.githubusercontent.com/garyongguanjie/learning-pyspark/main/data/titanic-train.csv
!wget https://raw.githubusercontent.com/garyongguanjie/learning-pyspark/main/data/titanic-test.csv

--2022-06-24 06:22:11--  https://raw.githubusercontent.com/garyongguanjie/learning-pyspark/main/data/titanic-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic-train.csv’

titanic-train.csv   100%[===================>]  58.89K  --.-KB/s    in 0.01s   

2022-06-24 06:22:11 (3.99 MB/s) - ‘titanic-train.csv’ saved [60302/60302]

--2022-06-24 06:22:12--  https://raw.githubusercontent.com/garyongguanjie/learning-pyspark/main/data/titanic-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('titanic-example').getOrCreate()

In [ ]:
import pyspark.sql.functions as F

# Reading Data

In [ ]:
df_train = spark.read.csv('titanic-train.csv',header=True, inferSchema=True)
df_test = spark.read.csv('titanic-test.csv',header=True, inferSchema=True)

In [ ]:
df_train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
df_test.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
only showing top 5 rows



In [ ]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df_train.select('Survived', 'Pclass', 'Age', 'Fare').describe().show()

+-------+-------------------+------------------+------------------+-----------------+
|summary|           Survived|            Pclass|               Age|             Fare|
+-------+-------------------+------------------+------------------+-----------------+
|  count|                891|               891|               714|              891|
|   mean| 0.3838383838383838| 2.308641975308642| 29.69911764705882| 32.2042079685746|
| stddev|0.48659245426485753|0.8360712409770491|14.526497332334035|49.69342859718089|
|    min|                  0|                 1|              0.42|              0.0|
|    max|                  1|                 3|              80.0|         512.3292|
+-------+-------------------+------------------+------------------+-----------------+



# EDA

In [ ]:
df_train.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [ ]:
df_train.groupBy('Survived').mean('Fare', 'Age').show()

+--------+------------------+------------------+
|Survived|         avg(Fare)|          avg(Age)|
+--------+------------------+------------------+
|       1| 48.39540760233917|28.343689655172415|
|       0|22.117886885245877| 30.62617924528302|
+--------+------------------+------------------+



In [ ]:
df_train.groupBy('Survived').pivot('Sex').count().show()

+--------+------+----+
|Survived|female|male|
+--------+------+----+
|       1|   233| 109|
|       0|    81| 468|
+--------+------+----+



In [ ]:
df_train.groupBy('Survived').pivot('Pclass').count().show()

+--------+---+---+---+
|Survived|  1|  2|  3|
+--------+---+---+---+
|       1|136| 87|119|
|       0| 80| 97|372|
+--------+---+---+---+



# Get percentage of null values

In [ ]:
df_train.select([(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in df_train.columns]).show()

+-----------+--------+------+----+---+-------------------+-----+-----+------+----+------------------+--------------------+
|PassengerId|Survived|Pclass|Name|Sex|                Age|SibSp|Parch|Ticket|Fare|             Cabin|            Embarked|
+-----------+--------+------+----+---+-------------------+-----+-----+------+----+------------------+--------------------+
|        0.0|     0.0|   0.0| 0.0|0.0|0.19865319865319866|  0.0|  0.0|   0.0| 0.0|0.7710437710437711|0.002244668911335578|
+-----------+--------+------+----+---+-------------------+-----+-----+------+----+------------------+--------------------+



In [ ]:
# view sql generated
[(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in df_train.columns]

[Column<'(count(CASE WHEN (isnan(PassengerId) OR (PassengerId IS NULL)) THEN PassengerId END) / count(1)) AS PassengerId'>,
 Column<'(count(CASE WHEN (isnan(Survived) OR (Survived IS NULL)) THEN Survived END) / count(1)) AS Survived'>,
 Column<'(count(CASE WHEN (isnan(Pclass) OR (Pclass IS NULL)) THEN Pclass END) / count(1)) AS Pclass'>,
 Column<'(count(CASE WHEN (isnan(Name) OR (Name IS NULL)) THEN Name END) / count(1)) AS Name'>,
 Column<'(count(CASE WHEN (isnan(Sex) OR (Sex IS NULL)) THEN Sex END) / count(1)) AS Sex'>,
 Column<'(count(CASE WHEN (isnan(Age) OR (Age IS NULL)) THEN Age END) / count(1)) AS Age'>,
 Column<'(count(CASE WHEN (isnan(SibSp) OR (SibSp IS NULL)) THEN SibSp END) / count(1)) AS SibSp'>,
 Column<'(count(CASE WHEN (isnan(Parch) OR (Parch IS NULL)) THEN Parch END) / count(1)) AS Parch'>,
 Column<'(count(CASE WHEN (isnan(Ticket) OR (Ticket IS NULL)) THEN Ticket END) / count(1)) AS Ticket'>,
 Column<'(count(CASE WHEN (isnan(Fare) OR (Fare IS NULL)) THEN Fare END) / c

# Fill in missing data

Unlike libraries such as xgboost and lgbm which accept na values,
unfortunately spark ml libraries currently do not accept na values. 

Hence we need to fill in them somehow. For this exercise we will fill them with reasonable defaults.

We can see that age, Cabin and Embarked has some na values.

Let's fill some na values here.

In [ ]:
avg_age = df_train.agg({'Age':'mean'}).withColumnRenamed("avg(Age)", "avg_age").collect()[0].avg_age

In [ ]:
df_train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
def fill_titanic_na(df):
    """
    Fill na values
    """
    return df.fillna({'Age':avg_age,'Embarked':'unknown','Fare':0.0})

In [ ]:
df_train = fill_titanic_na(df_train)
df_test = fill_titanic_na(df_test)

# Training

The first thing that we want to do is to split the data for validation.

In [ ]:
tr_df, val_df = df_train.randomSplit([0.8, 0.2])

Now we need to encode the data so that it can be trained in a usable format.

The first step is to convert all categorical columns into a number. We use the string indexer for this. This is similar to sklearn label encoder.

In [ ]:
tr_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
stringIndex = StringIndexer(inputCols=['Sex', 'Embarked'], 
                       outputCols=['SexNum', 'EmbNum'],handleInvalid='keep') # Set handle invalid to keep to dump unseen labels to unknown bucket.

stringIndex_model = stringIndex.fit(tr_df)

tr_df_1 = stringIndex_model.transform(tr_df)
tr_df_1.show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|SexNum|EmbNum|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   0.0|   0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   1.0|   1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|   1.0|   0.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   1.0|   0.0|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------

Next we need to convert the dataframe into a vector in order to pass it into our model. We do this using the vector assembler.

In [ ]:
feat_cols = ['Pclass','Age','SibSp','Parch','Fare','SexNum','EmbNum']
all_cols = feat_cols + ['Survived']

In [ ]:
vec_asmbl = VectorAssembler(inputCols=feat_cols, 
                           outputCol='features')

tr_df_1 = vec_asmbl.transform(tr_df_1)
tr_df_1.show(4, truncate=False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------+----------------------------------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|SexNum|EmbNum|features                          |
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+------+------+----------------------------------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |0.0   |0.0   |[3.0,22.0,1.0,0.0,7.25,0.0,0.0]   |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |1.0   |1.0   |[1.0,38.0,1.0,0.0,71.2833,1.0,1.0]|
|3          |1       |3  

Apply the same transformation to validation data

In [ ]:
val_df_1 = stringIndex_model.transform(val_df)
val_df_1 = vec_asmbl.transform(val_df_1)
val_df_1.show(4, truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+-----------------+-----+-----+-------+----+-----+--------+------+------+----------------------------------------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age              |SibSp|Parch|Ticket |Fare|Cabin|Embarked|SexNum|EmbNum|features                                |
+-----------+--------+------+-------------------------------------------------------+------+-----------------+-----+-----+-------+----+-----+--------+------+------+----------------------------------------+
|11         |1       |3     |Sandstrom, Miss. Marguerite Rut                        |female|4.0              |1    |1    |PP 9549|16.7|G6   |S       |1.0   |0.0   |[3.0,4.0,1.0,1.0,16.7,1.0,0.0]          |
|16         |1       |2     |Hewlett, Mrs. (Mary D Kingcome)                        |female|55.0             |0    |0    |248706 |16.0|null |S       |1.0   |0.0   |[2.0,55.0,0.

You might see some outputs like `(7,[0,1,4],[1.0,54.0,51.8625])` in the feature column. 
 
 Note this is to save space for sparse vectors (i.e. vectors with mostly zeros).

 They are in the format ```(number of columns,nonzero index,values)```

 Hence 
 `(7,[0,1,4],[1.0,54.0,51.8625])` = `[1.0,54.0,0,0,51.8625,0,0]`




In [ ]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='Survived', metricName='accuracy')

In [ ]:
gb = GBTClassifier(labelCol='Survived') # an estimator by default will use the 'features' column

model = gb.fit(tr_df_1) # Unlike sklearn's api here, .fit returns a transformer/model. (sklearn would returns self)
pred = model.transform(val_df_1)
evaluator.evaluate(pred)

0.828125

In [ ]:
pred.show(5)

+-----------+--------+------+--------------------+------+-----------------+-----+-----+--------+-------+-----+--------+------+------+--------------------+--------------------+--------------------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|SexNum|EmbNum|            features|       rawPrediction|         probability|prediction|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+--------+-------+-----+--------+------+------+--------------------+--------------------+--------------------+----------+
|         11|       1|     3|Sandstrom, Miss. ...|female|              4.0|    1|    1| PP 9549|   16.7|   G6|       S|   1.0|   0.0|[3.0,4.0,1.0,1.0,...|[0.15240133169779...|[0.57561614594244...|       0.0|
|         16|       1|     2|Hewlett, Mrs. (Ma...|female|             55.0|    0|    0|  248706|   16.0| null|       S|   1.0|   0.0|[2.0,55.0,0.0,0.0...|[0.86880539790

# Pipeline
The above process is abit messy and we should do it all in one shot just like sklearn's pipeline.

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[stringIndex,vec_asmbl,gb])

Let's just train on all data here

In [ ]:
model2 = pipeline.fit(df_train)

In [ ]:
# evaluation inaccurate here as we are training on all data
# pred2 = model2.transform(val_df)
# evaluator.evaluate(pred2)

# Make prediction on test data

In [ ]:
test_pred = model2.transform(df_test)

In [ ]:
test_pred = test_pred.withColumn('Survived',test_pred['prediction'].cast('integer'))\
    .select('PassengerId','Survived')

In [ ]:
test_pred.show(5)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|        892|       0|
|        893|       0|
|        894|       0|
|        895|       0|
|        896|       0|
+-----------+--------+
only showing top 5 rows



You can use `df.write.csv` to write the csv file. However it will write it into multiple csv files based on the number of nodes. You can use `.coalesce(1)` to combine them into 1 file before writing but it will still write it into a directory except that it has only 1 file in the directory.

For this case since the dataset is small we will convert it to pandas before writing it into a csv file.

In [ ]:
test_pred.toPandas().to_csv('submission.csv', index=False)

You can submit it on kaggle to see how well you score!

https://www.kaggle.com/competitions/titanic/overview

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Save the model

In [ ]:
model2.write().save('titanic_classification.model')

# Conclusion

This is a simple titanic tutorial. The score is not very high and there are a few ways to improve. Some ways are to fill null values with other methods, hyperparameter tuning, additional feature extraction.

Spark in general not as convenient as sklearn/xgboost/lgbm. Because writing transfomation in spark is not so straight forward as well as it's inability for the standard library to automatically handle null values.